## Import des bibliothèques nécessaires


In [28]:
import getpass, os, pymongo, pprintx
from llama_index.core import SummaryIndex, Settings, VectorStoreIndex
from llama_index.core import SimpleDirectoryReader
from llama_index.embeddings.mistralai import MistralAIEmbedding  # Adapté pour MistralAI
from llama_index.llms.mistralai import MistralAI  # Adapté pour MistralAI
from llama_index.core import StorageContext, load_index_from_storage
import nest_asyncio
from dotenv import load_dotenv
import os
import logging
import sys


In [35]:
import getpass, os, pymongo, pprint
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.core.settings import Settings
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.vector_stores import MetadataFilter, MetadataFilters, ExactMatchFilter, FilterOperator
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch

## Charger les variables d'environnement

&

## Vérifiez que la clé API est correctement chargée


In [29]:
load_dotenv()

api_key = os.getenv("MISTRAL_API_KEY")
print("MISTRAL_API_KEY:", api_key)


MISTRAL_API_KEY: f298tKx9ikNsDZs8sjM4W8TG3Oh8c22I


## Définition de la fonction pour la première approche avec SummaryIndex/VectoreIndex


In [15]:
# Application des correctifs pour l'environnement asynchrone
nest_asyncio.apply()

# Chargement des variables d'environnement
load_dotenv()

# Vérifiez que la clé API est correctement chargée
api_key = os.getenv("MISTRAL_API_KEY")
if not api_key:
    raise ValueError("You must provide an API key to use MistralAI. Set it in the .env file as MISTRAL_API_KEY.")
print("MISTRAL_API_KEY:", api_key)

# Configuration des logs
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Fonction d'initialisation et de persistance de la base de données vectorielle
def initialize_and_persist_vectorstore(dir_path, persist_dir):
    api_key = os.getenv("MISTRAL_API_KEY")

    if not os.path.exists(persist_dir):
        os.makedirs(persist_dir)

    if os.listdir(persist_dir):
        storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
        summary_index = load_index_from_storage(storage_context)
    else:
        reader = SimpleDirectoryReader(input_dir=dir_path)
        documents = reader.load_data()

        Settings.llm = MistralAI(max_tokens=10000, model="mistral-large-latest", api_key=api_key)  # Adapté pour MistralAI
        Settings.embed_model = MistralAIEmbedding(
            model_name="mistral-embed", 
            api_key=api_key,
            max_tokens=4000
        )
        # summary_index = summary_index.from_documents(documents)
        # summary_index.storage_context.persist(persist_dir=persist_dir)
        Vector_index = VectorStoreIndex.from_documents(documents)
        Vector_index.storage_context.persist(persist_dir=persist_dir)

    Settings.llm = MistralAI(max_tokens=10000, model="mistral-large-latest", api_key=api_key)  # Adapté pour MistralAI
    chat_engine = Vector_index.as_chat_engine(chat_mode="context")
    return chat_engine

MISTRAL_API_KEY: f298tKx9ikNsDZs8sjM4W8TG3Oh8c22I


In [36]:
persist_dir = "storage1"
storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
summary_index = load_index_from_storage(storage_context)
chat_engine = summary_index.as_chat_engine(chat_mode="context")
response = chat_engine.chat("Résumes le contexte")
print(response)

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embedding

In [38]:
chat_engine.chat_history

[ChatMessage(role=<MessageRole.USER: 'user'>, content='Résumes le contexte', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content="Le contexte fourni semble provenir d'un document lié à la Stratégie Nationale pour l'Intelligence Artificielle en France. Le document aborde les enjeux environnementaux et vise à préserver et consolider la souveraineté économique, technologique et politique du pays en mettant l'IA au service de l'économie et de la société.\n\nIl est mentionné que le document a été produit avec la participation financière de la direction du Numérique Responsable et le pôle Data/IA du Groupe La Poste. Un des principaux contributeurs est AFNOR, dont l'adresse email kassamarwane6@gmail.com est fournie.\n\nLe document aborde également les concepts d'apprentissage continu dans le contexte de l'IA, où les nouvelles données doivent être prises en compte. Il y a aussi une discussion sur les règles d'allocation pour les services numériques, avec deux

## Les paramètres de notre fonction


In [30]:
dir_path = "data"  # Chemin vers le répertoire contenant les textes du PDF
persist_dir = "storage"  # Chemin vers le répertoire de persistance


In [16]:
# Initialisation du moteur de chat
chat_engine = initialize_and_persist_vectorstore(dir_path, persist_dir)

# Exemple de requête au moteur de chat
query = "Quelle est la mission de l'entreprise ?"
response = chat_engine.chat(query)
print("Réponse du chatbot : ", response)

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: PO

# Commentaire:

- La réponse n'est pas satisfaisante, parceque la question est peut-être vague !
- notre objectif: Améliorer les réponse générées et s'assurer que notre DB vectorielle est bien faite !

Voici d'autres exemples de prompt pour vérifier la pertinences des informations récupérés !


In [17]:
response = chat_engine.chat("Tu me génère 20 questions pour évaluer l’utilisateur sur ses connaissance du document. format json brut")
print("Réponse du chatbot : ", response)

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/chat/completion

In [19]:
response = chat_engine.chat("Tu me génère 10 questions pour évaluer l’utilisateur sur ses connaissance du document, sous les questions doivent contenir (la question et les éléments de réponse, non pas les choix comme un qcm). format json brut")
print("Réponse du chatbot : ", response)

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/chat/completion

# Utilisation de la fonction de division sémantique pour diviser le texte en segments sémantiques:

_Le semantic splitting est utilisé pour diviser le texte en segments sémantiques, ce qui permet de mieux définir les nœuds dans la création du VectorStore avec LlamaIndex. Cela améliore la précision et la pertinence des résultats en regroupant les informations similaires dans des nœuds distincts._


In [34]:
from llama_index.core.node_parser import (
    SemanticSplitterNodeParser,
)

# Application des correctifs pour l'environnement asynchrone
nest_asyncio.apply()

# Chargement des variables d'environnement
load_dotenv()

# Vérifiez que la clé API est correctement chargée
api_key = os.getenv("MISTRAL_API_KEY")
if not api_key:
    raise ValueError("You must provide an API key to use MistralAI. Set it in the .env file as MISTRAL_API_KEY.")
print("MISTRAL_API_KEY:", api_key)

# Configuration des logs
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Fonction d'initialisation et de persistance de la base de données vectorielle
def initialize_and_persist_vectorstore1(dir_path, persist_dir):
    api_key = os.getenv("MISTRAL_API_KEY")

    if not os.path.exists(persist_dir):
        os.makedirs(persist_dir)

    if os.listdir(persist_dir):
        storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
        summary_index = load_index_from_storage(storage_context)
    else:
        reader = SimpleDirectoryReader(input_dir=dir_path)
        documents = reader.load_data()

        Settings.llm = MistralAI(max_tokens=10000, model="mistral-large-latest", api_key=api_key)  # Adapté pour MistralAI
        Settings.embed_model = MistralAIEmbedding(
            model_name="mistral-embed", 
            api_key=api_key,
            max_tokens=4000
        )
        splitter = SemanticSplitterNodeParser(
            buffer_size=1, breakpoint_percentile_threshold=95,embed_model=Settings.embed_model
        )
        nodes = splitter.get_nodes_from_documents(documents)
        summary_index = VectorStoreIndex(nodes)
        summary_index.storage_context.persist(persist_dir=persist_dir)

    Settings.llm = MistralAI(max_tokens=10000, model="mistral-large-latest", api_key=api_key)  # Adapté pour MistralAI
    chat_engine = summary_index.as_chat_engine(chat_mode="context")
    return chat_engine

MISTRAL_API_KEY: f298tKx9ikNsDZs8sjM4W8TG3Oh8c22I


In [32]:

dir_path = "data"  # Chemin vers le répertoire contenant les textes du PDF
persist_dir = "storage1"  # Chemin vers le répertoire de persistance / Changement du dossier de persistance

chat_engine = initialize_and_persist_vectorstore1(dir_path, persist_dir)

# Exemple de requête au moteur de chat
query = "Quelle est la mission de l'entreprise ?"
response = chat_engine.chat(query)
print("Réponse du chatbot : ", response)

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: PO

In [33]:
response = chat_engine.chat("Tu me résumes les concepts traités dans le document ?")
print("Réponse du chatbot : ", response)

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK

In [27]:
response = chat_engine.chat("Tu me génère 10 questions pour évaluer l’utilisateur sur ses connaissance du document, sous les questions doivent contenir (la question et les éléments de réponse, non pas les choix comme un qcm). format json brut")
print(response)

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 